Copyright (c) University of Strasbourg. All Rights Reserved.

# ORPose-Color
-------------------
**Self-supervision on Unlabelled OR Data for Multi-person 2D/3D Human Pose Estimation (MICCAI-2020)**

_Vinkle Srivastav, Afshin Gangi, Nicolas Padoy_

This repository contains the inference demo and evaluation scripts.

[![arXiv](https://img.shields.io/badge/arxiv-2007.08354-red)](https://arxiv.org/abs/2007.08354) 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CAMMA-public/ORPose-Color/blob/master/orpose_color_demo.ipynb)

**This demo notebook contains the inference scripts for the following models: _ORPose_fixed_8x_, _ORPose_fixed_10x_, and _ORPose_fixed_12x_**

<br/>

## Install dependencies and download the trained models
**Needed only for the colab demo. Please make sure to enable "GPU" using EDIT->Notebook settings.**

In [ ]:
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

# Install detectron2
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html 

In [ ]:
# download the trained models
!git clone https://github.com/CAMMA-public/ORPose-Color.git
%cd ORPose-Color

!wget https://s3.unistra.fr/camma_public/github/ORPose/models.zip
!unzip models.zip
!rm models.zip

## Imports

In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import numpy as np
import cv2
import random
from base64 import b64encode
from IPython.display import HTML
from PIL import Image
import glob
from tqdm import tqdm 
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from keypoints_3d import add_keypoints3d_config, draw_2d_keypoints, bgr2rgb, coco_to_h36_kps, images_to_video, progress_bar
import matplotlib.pyplot as plt

## Choose the model type and set the paths to the data and model directory

In [ ]:
# Paths and params
SCALE = 12 # 10, 12

MODEL_WEIGHTS = "models/orpose_fixed_{}x_model_final.pth".format(SCALE)
CONFIG_FILE = "configs/orpose_fixed_{}x.yaml".format(SCALE)
VIDEO_DIR = "data/mvor_seq1_x{}_color".format(SCALE)
    
H36_STATS_FILE = "data/h36stats.json"
OUTPUT_DIR = "output"
OUTPUT_VID_NAME = os.path.join(OUTPUT_DIR, "output.mp4")
IMG_FORMAT = "BGR"
IMG_RESIZE_SHAPE = (480, 640)

## Setup the config file and load the model weights

In [ ]:
cfg = get_cfg()
add_keypoints3d_config(cfg)
cfg.merge_from_file(CONFIG_FILE)

cfg.MODEL.WEIGHTS = MODEL_WEIGHTS
cfg.DATASETS.H36_STATS_PATH = H36_STATS_FILE
predictor = DefaultPredictor(cfg)

##  Inference demo on the video frames

In [ ]:
# read the files
files = sorted(glob.glob(VIDEO_DIR+"/*.png"))

# perform inference on video
print("inferece on video started")
# Initialize the progress-bar
bar = display(progress_bar(1, len(files)), display_id=True)
for index, (img_path) in enumerate(files):
    img_lr = utils.read_image(img_path, format="BGR")
    # Resize using bilinear interpolation
    img_sr, _ = T.apply_transform_gens([T.Resize(shape= IMG_RESIZE_SHAPE, interp=Image.BILINEAR)], img_lr)
    predictions = predictor(img_sr)
    predictions = predictions["instances"].to("cpu")
    
    # get boxes, keypoints2d and keypoints3d
    boxes = predictions.pred_boxes if predictions.has("pred_boxes") else None
    scores = predictions.scores if predictions.has("scores") else None
    keypoints2d_coco = predictions.pred_keypoints if predictions.has("pred_keypoints") else None 
    keypoints2d_h36 = predictions.pred_keypoints_h36_2d if predictions.has("pred_keypoints_h36_2d") else None
    keypoints3d_h36 = predictions.pred_keypoints_h36_3d if predictions.has("pred_keypoints_h36_3d") else None    
    
    # convert pytorch tensors to numpy
    boxes = boxes.tensor.numpy()
    keypoints2d_h36 = np.asarray(keypoints2d_h36)
    keypoints2d_coco = np.asarray(keypoints2d_coco)
    keypoints3d_h36 = np.asarray(keypoints3d_h36)
    for sc, box, kpts_2d in zip(scores, boxes, keypoints2d_coco):
        if sc > 0.5:
            draw_2d_keypoints(img_sr, coco_to_h36_kps(kpts_2d), style="h36", box=box)  
    cv2.imwrite(os.path.join(OUTPUT_DIR, f"{index:06d}" + ".png"), img_sr)
    bar.update(progress_bar(index + 1, len(files)))
print("inferece on video finished")

In [ ]:
# Convert the rendered images to video
images_to_video(OUTPUT_DIR, OUTPUT_VID_NAME)  

## Show the output video

In [ ]:
mp4 = open(OUTPUT_VID_NAME, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(""" <video width=400 controls>
         <source src="%s" type="video/mp4">
         </video> 
     """ % data_url
    )